In [2]:
# --- Before your go ----
# 1. Rename Assignment-03-###.ipynb where ### is your student ID.
# 2. The deadline of Assignment-03 is 23:59pm, 05-31-2023


# --- Explore HMM POS Taggers using Brown corpus ---
# In this assignment, you will explore two taggers for a Brown corpus.
# import your packages here

In [3]:

# Task 1 --- Load and explore your data ---
# 1). load train/test samples from Brown corpus files, brown-train.txt, brown-test.txt.
# 2). load all 12 tags from brown-tag.txt and print it out
# 3). counting how many sentences and words in both train and test datasets.
# 4). for each tag, counting how many words in train and test. e.g, tag1: [count_tr, count_te]

# Your code
def load_tags(file_path):
    tags = []
    with open(file_path, 'r') as file:
        for line in file:
            if (line.strip() != '.'):
                tags.append(line.strip())
                print(line.strip())
    return tags

def load_file(file_path):
    dictionary = {'ADJ':0,'ADP':0,'ADV':0,'CONJ':0,'DET':0,'NOUN':0,'NUM':0,'PRON':0,'PRT':0,'VERB':0,'X':0}
    sent_count = 0
    word_count = 0 
    with open(file_path, 'r') as file:
        for line in file:
            split_line = line.split()
            if (len(split_line) == 1 and split_line[0][:4] == 'b100'):
                sent_count += 1
            elif (len(split_line) >= 2 and split_line[1] != '.'):
                word_count += 1
                dictionary[split_line[1]] += 1
    info = []
    info.append(word_count)
    info.append(sent_count)
    info.append(dictionary)
    return info

tags = load_tags('brown-tag.txt')

train_info = load_file('brown-train.txt');
test_info = load_file('brown-test.txt');

print("train datasets sentences: {}, words: {}".format(train_info[1], train_info[0]))
print("test datasets: sentences: {}, words: {}".format(test_info[1], test_info[0]))
for i in range(0, 11):
    print("tag: {}, the num of words: {}".format(tags[i], train_info[2][tags[i]] + test_info[2][tags[i]]))

ADJ
ADP
ADV
CONJ
DET
NOUN
NUM
PRON
PRT
VERB
X
train datasets sentences: 45800, words: 810604
test datasets: sentences: 11540, words: 203023
tag: ADJ, the num of words: 83721
tag: ADP, the num of words: 144766
tag: ADV, the num of words: 56239
tag: CONJ, the num of words: 38151
tag: DET, the num of words: 137019
tag: NOUN, the num of words: 275558
tag: NUM, the num of words: 14874
tag: PRON, the num of words: 49334
tag: PRT, the num of words: 29829
tag: VERB, the num of words: 182750
tag: X, the num of words: 1386


In [4]:
# Task 2 --- Build a baseline method, namely, the most frequent tagger ---
#     If you can recall, we introduced a strong baseline method (See Dan's book in 
# https://web.stanford.edu/~jurafsky/slp3/ed3book_jan72023.pdf Page 164.),
#     where we label each word by using the most frequent-used tag associated with it.
# 1). find the most frequent class label for each word in the training data.
#     For example, {tr_word_1:tag_1,tr_word_2:tag_2,...}
# 2). use your built method to predict tags for both train and test datasets.
#     You should print out two values: the accuracies of train and test samples.
#     You would expect that the accuracy on train will be > 9.0 (but never = 1.0) and higher than on test.

# Notice: since there are unknown words in test samples. 
#  Following ways could handle this (choose one or create your own): 
#  1). mark all words that appear only once in the data with a "UNK-x" tag
#  2). tag every out-of-vocabulary word with the majority tag among all training samples.
#  3). find more methods in https://github.com/Adamouization/POS-Tagging-and-Unknown-Words

# Your code
from collections import defaultdict
from collections import Counter

def load_file(file_path):
    dictionary = []
    with open(file_path, 'r') as file:
        training_words = []
        training_tags = []
        for line in file:
            split_line = line.split()
            if (len(split_line) >= 2 and split_line[1] != '.'):
                training_words.append(split_line[0])
                training_tags.append(split_line[1])
    dictionary.append(training_words)   
    dictionary.append(training_tags)   
    return dictionary

def label_words_with_most_frequent_tag(words, tags):
    word_tag_dict = defaultdict(list)
    for word, tag in zip(words, tags):
        word_tag_dict[word].append(tag)
    word_most_frequent_tag_dict = {}
    # Find the most frequent tag for each word
    for word, tag_list in word_tag_dict.items():
        tag_counts = Counter(tag_list)
        most_frequent_tag = tag_counts.most_common(1)[0][0]
        word_most_frequent_tag_dict[word] = most_frequent_tag
    return word_most_frequent_tag_dict

training_words = load_file("brown-train.txt")[0]
training_tags = load_file("brown-train.txt")[1]

word_most_frequent_tag_dict = label_words_with_most_frequent_tag(training_words, training_tags)

i = 0
for word, tag in word_most_frequent_tag_dict.items():
    if (i < 20):
        print(f"{word}: {tag}")
        i+=1
    else:
        break

Mr.: NOUN
Podger: NOUN
had: VERB
thanked: VERB
him: PRON
gravely: ADV
and: CONJ
now: ADV
he: PRON
made: VERB
use: NOUN
of: ADP
the: DET
advice: NOUN
But: CONJ
there: PRT
seemed: VERB
to: PRT
be: VERB
some: DET


In [5]:
# 2). use your built method to predict tags for both train and test datasets.
#     You should print out two values: the accuracies of train and test samples.
#     You would expect that the accuracy on train will be > 9.0 (but never = 1.0) and higher than on test.

# Notice: since there are unknown words in test samples. 
#  Following ways could handle this (choose one or create your own): 
#  1). mark all words that appear only once in the data with a "UNK-x" tag
#  2). tag every out-of-vocabulary word with the majority tag among all training samples.
#  3). find more methods in https://github.com/Adamouization/POS-Tagging-and-Unknown-Words

def accurancy(file_path, word_most_frequent_tag_dict, tagDict):
    dictionary = []
    with open(file_path, 'r') as file:
        count = 0
        right = 0
        for line in file:
            split_line = line.split()
            if (len(split_line) >= 2 and split_line[1] != '.'):
                count += 1
                if ((split_line[0] in word_most_frequent_tag_dict) and word_most_frequent_tag_dict[split_line[0]] == split_line[1]):
                    right += 1
                elif (split_line[0] not in word_most_frequent_tag_dict):
                    max_value = max(tagDict.values())
                    max_keys = [key for key, value in tagDict.items() if value == max_value]
                    word_most_frequent_tag_dict[split_line[0]] = max_keys
    return right / count

tagDict = {}
for i in range(0, 11):
    tagDict[tags[i]] = train_info[2][tags[i]]

print("the accurancy of train set: {}".format(accurancy("brown-train.txt", word_most_frequent_tag_dict, tagDict)))
print("the accurancy of test set: {}".format(accurancy("brown-test.txt", word_most_frequent_tag_dict, tagDict)))

the accurancy of train set: 0.951018006326147
the accurancy of test set: 0.9201962339242352


In [6]:
# Task 3 --- Build an HMM tagger ---
# 1) You should use nltk.tag.HiddenMarkovModelTagger to build an HMM tagger.
#    It has parameters: symbols, states, transitions, outputs, priors, transform (ignore it).
#    Specify these parameters properly. For example, you can use MLE to estimate transitions, outputs and priors.
#    That is, MLE to estimate matrix A (transition matrix), and matrix B (output probabilites) (See. Page 8.4.3)
# 2) After build your model, report both the accuracy of HMM tagger for train samples and test samples.
# 
# 3) Compared with your baseline method, discuss that why your HMM tagger is better/worse than baseline method.

# Notice: You may also need to handle unknown words just like Task 2.

# Your code
from nltk.tag import HiddenMarkovModelTagger

# Define the symbols (observations)
symbols = list(word_most_frequent_tag_dict.keys())
# Define the states (POS tags)
states = tags
print(states)
print(symbols)

['ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']
['Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', 'But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'opinion', 'as', 'how', 'far', 'board', 'should', 'go', 'whose', 'it', 'follow', 'Such', 'an', 'instrument', 'is', 'expected', 'especially', 'useful', 'if', 'could', 'used', 'measure', 'elasticity', 'heavy', 'pastes', 'such', 'printing', 'inks', 'paints', 'adhesives', 'molten', 'plastics', 'bread', 'dough', 'for', 'related', 'those', 'various', 'properties', 'termed', 'length', 'shortness', 'spinnability', 'etc.', 'which', 'are', 'usually', 'judged', 'by', 'subjective', 'methods', 'at', 'present', 'My', 'future', 'plans', 'become', 'a', 'language', 'teacher', 'We', 'ran', 'east', 'about', 'half', 'mile', 'before', 'we', 'turned', 'back', 'road', 'panting', 'from', 'effort', 'soaked', 'with', 'sweat', 'After', 'television', 'La', 'Dolce', 

In [7]:
# get transition
# 马尔可夫假设
def get_transition(file_path):
    with open(file_path, 'r') as file:
        line = file.readline()
        res = []
        while line:
            split_line = line.split()
            if (len(split_line) == 1 and split_line[0][:4] == 'b100'):
                tmp = []
                line = file.readline()
                while (line and len(line.split()) >= 2):
                    if line.split()[1] != '.':
                        tmp.append((line.split()[0], line.split()[1]))
                    line = file.readline()
                res.append(tmp)
            line = file.readline()
    return res

train_data = get_transition('brown-train.txt')
transition_counts = defaultdict(lambda: defaultdict(int))

for sentence in train_data:
    for i in range(len(sentence) - 1):
        prev_pos, current_pos = sentence[i][1], sentence[i + 1][1]
        transition_counts[prev_pos][current_pos] += 1

transition_matrix = {}
for prev_pos, counts in transition_counts.items():
    total_count = sum(counts.values())
    transition_probabilities = {current_pos: count / total_count for current_pos, count in counts.items()}
    transition_matrix[prev_pos] = transition_probabilities
    
print(transition_matrix)

{'NOUN': {'NOUN': 0.20504457718422966, 'VERB': 0.21025877277938612, 'ADP': 0.3038630069574959, 'PRON': 0.03660930702502919, 'ADJ': 0.021935220425407163, 'CONJ': 0.09696519157980703, 'ADV': 0.04328410560316621, 'X': 0.0006491568813246989, 'PRT': 0.024390499274933645, 'DET': 0.04350921645717397, 'NUM': 0.013490945832046363}, 'VERB': {'VERB': 0.1985569575262129, 'PRON': 0.061908654700550916, 'NOUN': 0.10641549671228008, 'PRT': 0.06956459925359872, 'DET': 0.17460813932823885, 'CONJ': 0.021084059001244004, 'ADV': 0.11199573484983118, 'ADP': 0.18305313666252, 'ADJ': 0.06271192464901368, 'NUM': 0.009774302470232806, 'X': 0.00032699484627687935}, 'PRON': {'ADV': 0.06208336679784756, 'VERB': 0.7583594356544321, 'DET': 0.023746419296977486, 'ADP': 0.06636682461917383, 'PRT': 0.02746767327925468, 'NOUN': 0.015393676545391267, 'CONJ': 0.019355875030118062, 'ADJ': 0.01081573099884882, 'PRON': 0.014938559151875351, 'NUM': 0.0013921237919310363, 'X': 8.031483414986748e-05}, 'ADV': {'CONJ': 0.02853053

In [8]:
# Define the outputs (probabilities of generating symbols from states)
def get_outputs(file_path):
    dictionary = {'ADJ':{},'ADP':{},'ADV':{},'CONJ':{},'DET':{},'NOUN':{},'NUM':{},'PRON':{},'PRT':{},'VERB':{},'X':{}} 
    with open(file_path, 'r') as file:
        for line in file:
            split_line = line.split()
            if (len(split_line) >= 2 and split_line[1] != '.'):
                if split_line[0] not in dictionary[split_line[1]]:
                    dictionary[split_line[1]][split_line[0]] = 1
                else:
                    dictionary[split_line[1]][split_line[0]] += 1
    return dictionary

outputs = get_outputs('brown-train.txt')
for i in range(0, 11):
    outputs[tags[i]] = {key: value/train_info[0] for key, value in outputs[tags[i]].items()} 
outputs

{'ADJ': {'useful': 5.7981455803326905e-05,
  'heavy': 0.00010732737563594554,
  'such': 0.0008931611489703974,
  'molten': 1.233647995815466e-06,
  'various': 0.00019491638333884363,
  'subjective': 2.0972015928862923e-05,
  'future': 7.525252774474343e-05,
  'harmless': 4.934591983261864e-06,
  'Gray': 8.635535970708261e-06,
  'North': 8.882265569871356e-05,
  'chief': 5.9215103799142365e-05,
  'cold': 0.00013940222352714767,
  'clear': 0.00019368273534302817,
  'own': 0.000741422445485095,
  'evil': 3.577579187864851e-05,
  'English': 0.00010979467162757647,
  'recent': 0.00016160788745182606,
  'interesting': 7.771982373637436e-05,
  'vast': 5.551415981169597e-05,
  'amorphous': 4.934591983261864e-06,
  'substantial': 6.044875179495783e-05,
  'extraordinary': 2.9607551899571182e-05,
  'dull': 2.837390390375572e-05,
  'upper': 6.16823997907733e-05,
  'Jewish': 8.018711972800528e-05,
  'last': 0.0005785809100374536,
  'eight-and-a-half-foot': 1.233647995815466e-06,
  'late': 0.0001295

In [9]:
# Define the priors (initial probabilities of starting from each state)
a = sum(tagDict.values())
priors = {key: value/a for key, value in tagDict.items()}
priors

{'ADJ': 0.08263591099969898,
 'ADP': 0.14279722281163182,
 'ADV': 0.05522425253267933,
 'CONJ': 0.03757074971256002,
 'DET': 0.13498329640613665,
 'NOUN': 0.2719589343255153,
 'NUM': 0.01470631775811617,
 'PRON': 0.048922778570053935,
 'PRT': 0.02947061697203567,
 'VERB': 0.1803581033402253,
 'X': 0.001371816571346798}

In [10]:
# Build the HMM tagger
from nltk.probability import ConditionalFreqDist, ConditionalProbDist, MLEProbDist, DictionaryProbDist, DictionaryConditionalProbDist
from nltk.tag.hmm import HiddenMarkovModelTagger
import numpy as np
# transition = [[v for k, v in d.items()] for d in transition_matrix.values()]
# output = [[v for k, v in d.items()] for d in outputs.values()]
# prior = np.array(list(priors.values()))

tagger = HiddenMarkovModelTagger(symbols=symbols, states=states,
                                 transitions=ConditionalProbDist(transition_matrix, MLEProbDist), 
                                 outputs=ConditionalProbDist(outputs,MLEProbDist),
                                                             priors=DictionaryProbDist(priors))
tagger

<HiddenMarkovModelTagger 11 states and 56049 output symbols>

In [11]:
tagger.tag(["the"])

AttributeError: 'dict' object has no attribute 'freq'